<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
    </a>
</p>

# Final Project: Advanced SQL Techniques

## Objectives

After completing this lab, you will be able to:

*   Use joins to query data from multiple tables

*   Create and query views

*   Write and run stored procedures

*   Use transactions


## Scenario

In this project, you will work with three datasets that are available on the City of Chicago's Data Portal:

*    Socioeconomic indicators in Chicago
*    Chicago public schools
*    Chicago crime data

You must download each dataset, create a table for each one, and load the appropiate dataset through the Db2 console. If you have already completed the Hands on Lab: Joins, you can reuse the tables you created for that hands-on lab. However, you should not reuse similar tables with other names from other exercises or labs, as they may not create the correct results.

Important note:

If you have not yet downloaded the three datasets from the City of Chicago's Data Portal, created the required tables, and loaded the data, please follow the instructions in this section.



### City of Chicago Datasets
    Socioeconomic indicators in Chicago

This dataset contains a selection of six socioeconomic indicators of public health significance and a "hardship index", for each Chicago community area, for the years 2008 – 2012. A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2

    Chicago public schools

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t

    Chicago crime data

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days. A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at: https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2

In [94]:
import urllib.request
from urllib.request import urlopen
import ssl
import json
import csv
import sqlite3
import pandas as pd

ssl._create_default_https_context = ssl._create_unverified_context

In [95]:
import csv, sqlite3

con = sqlite3.connect("RealWorldData.db")
cur = con.cursor()

%load_ext sql
%sql sqlite:///RealWorldData.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [96]:
df_se = pd.read_csv('https://data.cityofchicago.org/resource/jcxq-k9xf.csv')
df_se.to_sql("chicago_socioeconomic", con, if_exists='replace', index=False,method="multi")

78

In [97]:
df_ps = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df_ps.to_sql("chicago_public_schools", con, if_exists='replace')

566

In [98]:
df_cr = pd.read_json("https://data.cityofchicago.org/resource/ijzp-q8t2.json")
df_cr = df_cr.drop(columns = ["location"]) # need to remove any dictionaries! 
df_cr.to_sql("chicago_crimes", con, if_exists='replace', index=False)

1000

In [99]:
# First, find out what data is in each dataframe
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%sql SELECT * FROM chicago_socioeconomic limit 3

 * sqlite:///RealWorldData.db
Done.


ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0


In [100]:
%sql SELECT * FROM chicago_public_schools limit 3

 * sqlite:///RealWorldData.db
Done.


index,School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,Collaborative_Name,Adequate_Yearly_Progress_Made_,Track_Schedule,CPS_Performance_Policy_Status,CPS_Performance_Policy_Level,HEALTHY_SCHOOL_CERTIFIED,Safety_Icon,SAFETY_SCORE,Family_Involvement_Icon,Family_Involvement_Score,Environment_Icon,Environment_Score,Instruction_Icon,Instruction_Score,Leaders_Icon,Leaders_Score,Teachers_Icon,Teachers_Score,Parent_Engagement_Icon,Parent_Engagement_Score,Parent_Environment_Icon,Parent_Environment_Score,AVERAGE_STUDENT_ATTENDANCE,Rate_of_Misconducts__per_100_students_,Average_Teacher_Attendance,Individualized_Education_Program_Compliance_Rate,Pk_2_Literacy__,Pk_2_Math__,Gr3_5_Grade_Level_Math__,Gr3_5_Grade_Level_Read__,Gr3_5_Keep_Pace_Read__,Gr3_5_Keep_Pace_Math__,Gr6_8_Grade_Level_Math__,Gr6_8_Grade_Level_Read__,Gr6_8_Keep_Pace_Math_,Gr6_8_Keep_Pace_Read__,Gr_8_Explore_Math__,Gr_8_Explore_Read__,ISAT_Exceeding_Math__,ISAT_Exceeding_Reading__,ISAT_Value_Add_Math,ISAT_Value_Add_Read,ISAT_Value_Add_Color_Math,ISAT_Value_Add_Color_Read,Students_Taking__Algebra__,Students_Passing__Algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),Net_Change_EXPLORE_and_PLAN,11th Grade Average ACT (2011),Net_Change_PLAN_and_ACT,College_Eligibility__,Graduation_Rate__,College_Enrollment_Rate__,COLLEGE_ENROLLMENT,General_Services_Route,Freshman_on_Track_Rate__,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police_District,Location
0,610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf,Fullerton Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Very Strong,99.0,Very Strong,99,Strong,74.0,Strong,66.0,Weak,65,Strong,70,Strong,56,Average,47,96.00%,2.0,96.40%,95.80%,80.1,43.3,89.6,84.9,60.7,62.6,81.9,85.2,52,62.4,66.3,77.9,69.7,64.4,0.2,0.9,Yellow,Green,67.1,54.5,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,813,33,NDA,1171699.458,1915829.428,41.92449696,-87.64452163,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
1,610281,Adam Clayton Powell Paideia Community Academy Elementary School,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610281.pdf,Skyway Elementary Network,SOUTH SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,54.0,Strong,66,Strong,74.0,Very Strong,84.0,Weak,63,Strong,76,Weak,46,Average,50,95.60%,15.7,95.30%,100.00%,62.4,51.7,21.9,15.1,29,42.8,38.5,27.4,44.8,42.7,14.1,34.4,16.8,16.5,0.7,1.4,Green,Green,17.2,27.3,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,521,46,NDA,1196129.985,1856209.466,41.76032435,-87.55673627,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
2,610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610185.pdf,Midway Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 2,No,Strong,61.0,NDA,NDA,Average,50.0,Weak,36.0,Weak,NDA,NDA,NDA,Average,47,Weak,41,95.70%,2.3,94.70%,98.30%,53.7,26.6,38.3,34.7,43.7,57.3,48.8,39.2,46.8,44,7.5,21.9,18.3,15.5,-0.9,-1.0,Red,Red,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,1324,44,NDA,1148427.165,1851012.215,41.74711093,-87.73170248,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"


In [101]:
%sql SELECT * FROM chicago_crimes limit 3

 * sqlite:///RealWorldData.db
Done.


id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,:@computed_region_awaf_s7ux,:@computed_region_6mkv_f3dw,:@computed_region_vrxf_vc4k,:@computed_region_bdys_3d7i,:@computed_region_43wa_7qmu,:@computed_region_rpca_8um6,:@computed_region_d9mm_jgwp,:@computed_region_d3ds_rm58,:@computed_region_8hcu_yrd4
13767269,JJ171824,2025-03-02 00:00:00,006XX W 63RD ST,0820,THEFT,$500 AND UNDER,APARTMENT,0,0,723,7,20,68,06,1173239.0,1863118.0,2025,2025-03-09T15:41:23.000,41.779818631,-87.640427625,19.0,21559.0,66.0,435.0,2.0,11.0,17.0,135.0,20.0
13768046,JJ172698,2025-03-02 00:00:00,019XX W CORTLAND ST,0850,THEFT,ATTEMPT THEFT,STREET,0,0,1434,14,32,22,06,1163305.0,1912659.0,2025,2025-03-09T15:41:23.000,41.915977992,-87.675455706,16.0,21560.0,23.0,179.0,40.0,4.0,7.0,192.0,32.0
13764740,JJ167679,2025-03-02 00:00:00,040XX S PRAIRIE AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,0,1,213,2,3,38,08B,1178707.0,1878256.0,2025,2025-03-09T15:41:23.000,41.821236024,-87.619920712,12.0,4301.0,4.0,163.0,9.0,36.0,24.0,102.0,8.0


Write and execute a SQL query to list the school names, community names and average attendance for communities with a hardship index of 98.

In [102]:
# Both Public School and the Socioeconomic dataframes have community area name
# The public school df community area name is CAPITAL letters, and needs to be lowered.
# so the join matches!

%sql UPDATE chicago_socioeconomic SET community_area_name = LOWER(community_area_name)

%sql UPDATE chicago_public_schools SET COMMUNITY_AREA_NAME = LOWER(COMMUNITY_AREA_NAME)

 * sqlite:///RealWorldData.db
78 rows affected.
 * sqlite:///RealWorldData.db
566 rows affected.


[]

In [103]:
# Left join socioeconomic ONTO public schools

%sql SELECT chicago_public_schools.COMMUNITY_AREA_NAME, \
chicago_public_schools.AVERAGE_STUDENT_ATTENDANCE \
FROM chicago_public_schools LEFT JOIN chicago_socioeconomic \
ON chicago_public_schools.COMMUNITY_AREA_NAME = chicago_socioeconomic.community_area_name \
limit 10

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,AVERAGE_STUDENT_ATTENDANCE
lincoln park,96.00%
south shore,95.60%
ashburn,95.70%
new city,95.50%
armour square,93.30%
albany park,97.00%
north center,96.30%
west town,94.70%
north center,92.70%
albany park,96.40%


Write and execute a SQL query to list all crimes that took place at a school. Include case number, crime type and community name.

In [104]:
%sql SELECT cc.CASE_NUMBER, cc.PRIMARY_TYPE, cp.COMMUNITY_AREA_NAME \
FROM chicago_crimes cc , chicago_public_schools cp \
LEFT JOIN chicago_public_schools \
on cp.COMMUNITY_AREA_NUMBER = cc.community_area \
WHERE cc.LOCATION_DESCRIPTION LIKE '%School%' \
LIMIT 10

 * sqlite:///RealWorldData.db
Done.


case_number,primary_type,COMMUNITY_AREA_NAME
JJ166308,BATTERY,lincoln park
JJ166308,BATTERY,south shore
JJ166308,BATTERY,ashburn
JJ166308,BATTERY,new city
JJ166308,BATTERY,armour square
JJ166308,BATTERY,albany park
JJ166308,BATTERY,north center
JJ166308,BATTERY,west town
JJ166308,BATTERY,north center
JJ166308,BATTERY,albany park


Write and execute a SQL statement to create a view showing the columns listed in the following table, with new column names as shown in the second column.

In [105]:
%sql CREATE VIEW chicago_summary(School_Name,Safety_Rating,Family_Rating,Environment_Rating,Instruction_Rating,Leaders_Rating,Teachers_Rating) \
AS SELECT NAME_OF_SCHOOL,Safety_Icon, Family_Involvement_Icon, Environment_Icon,Instruction_Icon, Leaders_Icon, Teachers_Icon \
FROM chicago_public_schools;

 * sqlite:///RealWorldData.db
(sqlite3.OperationalError) view chicago_summary already exists
[SQL: CREATE VIEW chicago_summary(School_Name,Safety_Rating,Family_Rating,Environment_Rating,Instruction_Rating,Leaders_Rating,Teachers_Rating) AS SELECT NAME_OF_SCHOOL,Safety_Icon, Family_Involvement_Icon, Environment_Icon,Instruction_Icon, Leaders_Icon, Teachers_Icon FROM chicago_public_schools;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [106]:
%sql select * from chicago_summary limit 10

 * sqlite:///RealWorldData.db
Done.


School_Name,Safety_Rating,Family_Rating,Environment_Rating,Instruction_Rating,Leaders_Rating,Teachers_Rating
Abraham Lincoln Elementary School,Very Strong,Very Strong,Strong,Strong,Weak,Strong
Adam Clayton Powell Paideia Community Academy Elementary School,Average,Strong,Strong,Very Strong,Weak,Strong
Adlai E Stevenson Elementary School,Strong,NDA,Average,Weak,Weak,NDA
Agustin Lara Elementary Academy,Average,Average,Average,Weak,Weak,Average
Air Force Academy High School,Average,Strong,Strong,Average,Weak,Average
Albany Park Multicultural Academy,Strong,Weak,Strong,Strong,Weak,Average
Albert G Lane Technical High School,Very Strong,NDA,Strong,Average,Weak,NDA
Albert R Sabin Elementary Magnet School,Strong,NDA,Weak,Very Weak,Weak,NDA
Alcott High School for the Humanities,Strong,NDA,Strong,Average,Weak,NDA
Alessandro Volta Elementary School,Average,Strong,Weak,Weak,Weak,Average


Write and execute a SQL statement that returns just the school name and leaders rating from the view.

In [107]:
%sql select School_Name, Leaders_Rating from chicago_summary limit 10

 * sqlite:///RealWorldData.db
Done.


School_Name,Leaders_Rating
Abraham Lincoln Elementary School,Weak
Adam Clayton Powell Paideia Community Academy Elementary School,Weak
Adlai E Stevenson Elementary School,Weak
Agustin Lara Elementary Academy,Weak
Air Force Academy High School,Weak
Albany Park Multicultural Academy,Weak
Albert G Lane Technical High School,Weak
Albert R Sabin Elementary Magnet School,Weak
Alcott High School for the Humanities,Weak
Alessandro Volta Elementary School,Weak


Write the structure of a query to create or replace a stored procedure called UPDATE_LEADERS_SCORE that takes a in_School_ID parameter as an integer and a in_Leader_Score parameter as an integer. Don't forget to use the #SET TERMINATOR statement to use the @ for the CREATE statement terminator.

In [108]:
# SET TERMINATOR @
CREATE PROCEDURE UPDATE_LEADERS_SCORE(IN in_School_ID INTEGER, IN in_Leader_Score INTEGER)
BEGIN
 UPDATE CHICAGO_PUBLIC_SCHOOLS
 SET Leaders_Score = in_Leader_Score
 WHERE School_ID = in_School_ID ;

IF in_Leader_Score >0 AND in_Leader_Score <20
    THEN UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Icon ='Very Weak'
    WHERE School_ID = in_School_ID;
ELSEIF in_Leader_Score < 40
    THEN UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Icon ='Weak'
    WHERE School_ID = in_School_ID;
ELSEIF in_Leader_Score < 60
    THEN UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Icon ='Average'
    WHERE School_ID = in_School_ID;
ELSEIF in_Leader_Score < 80
    THEN UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Icon ='Strong'
    WHERE School_ID = in_School_ID;
ELSEIF in_Leader_Score < 100
    THEN UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Icon ='Very Strong'
    WHERE School_ID = in_School_ID;
ELSE  
   ROLLBACK;
END IF;
END@

SyntaxError: invalid syntax (4012461787.py, line 2)